Github
https://github.com/Shadowjumper3000/ML-fundamentals-2025

# Bike Rental Analysis
This analysis explores patterns in bike rental data to understand key factors influencing rental behavior. The data is sourced from the UCI Machine Learning Repository and contains hourly rental data spanning two years. The analysis is structured as follows:


In [ ]:
# Check if running in Kaggle environment
import os
IN_KAGGLE = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if IN_KAGGLE:
    # Kaggle-specific paths
    data_path = '/kaggle/input/bike-data/CapitalBikeSharing.csv'
    checkpoint_dir = '/kaggle/working/models'
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Create output directory for downloading models
    os.makedirs('/kaggle/working/output', exist_ok=True)

    print("Running in Kaggle environment.")
    print("Data will be loaded from:", data_path)
    print("Models will be saved to:", checkpoint_dir)
else:
    # Local paths
    data_path = '../data/hour.csv'
    checkpoint_dir = '../models/checkpoints'
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("Running in local environment.")
    print("Data will be loaded from:", data_path)
    print("Models will be saved to:", checkpoint_dir)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for better visualizations
plt.style.use('default')
sns.set_theme()

# Load the dataset
hour_data = pd.read_csv(data_path)
print("Data loaded from" + data_path)

## 1. Initial Data Exploration
Let's examine the basic structure and statistics of our dataset.
AI was used to write the functions for the analysis.

In [ ]:
# Initial Data Exploration
print("Dataset Shape:", hour_data.shape)
print("\nDataset Info:")
print(hour_data.info())
print("\nDescriptive Statistics:")
print(hour_data.describe())

### 1.1 Target Variable Analysis
Analyzing the distribution of bike rentals (cnt) to understand the general rental patterns.

In [ ]:
# Target Variable Analysis (cnt)
plt.figure(figsize=(10, 6))
sns.histplot(hour_data['cnt'], kde=True)
plt.title('Distribution of Bike Rentals (cnt)')
plt.xlabel('Number of Rentals')
plt.ylabel('Frequency')
print("\nSkewness of cnt:", hour_data['cnt'].skew())

### 1.2 Temporal Pattern Analysis
Examining how rental patterns vary by hour and season.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Hour analysis
plt.figure(figsize=(12, 6))
sns.boxplot(x='hr', y='cnt', data=hour_data)
plt.title('Hourly Rental Pattern')
plt.xlabel('Hour of Day')
plt.ylabel('Number of Rentals')
plt.show()

# Rentals per year
# Calculate total rentals per year - safer approach
yearly_rentals = hour_data.groupby('yr')['cnt'].sum().reset_index()
yearly_rentals['year'] = yearly_rentals['yr'].map({0: '2011', 1: '2012'})

plt.figure(figsize=(8, 5))
ax = sns.barplot(x='year', y='cnt', data=yearly_rentals, 
                color=sns.color_palette("Blues_d")[3])
plt.title('Total Rentals Per Year')
plt.xlabel('Year')
plt.ylabel('Total Rentals')

# Add value labels on top of bars
for p in ax.patches:
    ax.annotate(f'{int(p.get_height()):,}', 
               (p.get_x() + p.get_width() / 2., p.get_height()),
               ha='center', va='bottom',
               xytext=(0, 5),
               textcoords='offset points')
plt.show()

# Rentals per month
# Calculate total rentals per month - more robust approach
monthly_rentals = hour_data.groupby('mnth')['cnt'].sum().reset_index()

plt.figure(figsize=(10, 6))
ax = sns.barplot(x='mnth', y='cnt', data=monthly_rentals, 
                color=sns.color_palette("Blues_d")[3])
plt.title('Total Rentals Per Month', pad=15)
plt.xlabel('Month')
plt.ylabel('Total Rentals')
plt.xticks(ticks=range(12), 
          labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], 
          rotation=45)

# Add value labels on top of bars
for p in ax.patches:
    ax.annotate(f'{int(p.get_height()):,}', 
               (p.get_x() + p.get_width() / 2., p.get_height()),
               ha='center', va='bottom',
               xytext=(0, 5),
               textcoords='offset points')

plt.tight_layout()
plt.show()

In [ ]:
from scipy.stats import chi2_contingency
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Create a cross-tabulation between season and month
season_month_crosstab = pd.crosstab(
    hour_data['season'],
    hour_data['mnth']
)

# Perform chi-square test for independence
chi2, p, dof, expected = chi2_contingency(season_month_crosstab)

print(f"Chi2: {chi2:.2f}")
print(f"p-value: {p:.10f}")
print(f"DOF: {dof}")
print(f"Hypothesis: {'Rejected' if p < 0.05 else 'Not rejected'}")

# Compare correlation with target variable
print("\nCorrelation with target variable:")
print(f"Season-cnt correlation: {hour_data['season'].corr(hour_data['cnt']):.4f}")
print(f"Month-cnt correlation: {hour_data['mnth'].corr(hour_data['cnt']):.4f}")

# Check distribution of values
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.countplot(x='season', data=hour_data)
plt.title('Season Distribution')

plt.subplot(1, 2, 2)
sns.countplot(x='mnth', data=hour_data)
plt.title('Month Distribution')
plt.tight_layout()
plt.show()

As the 'Seasons' and 'mnth's are correlated, we will drop the 'mnth' column as this is less correlated with 'cnt' than 'seasons'.

In [ ]:
# Drop the mnth column as it's highly correlated with season
hour_data = hour_data.drop(columns=['mnth'])
print("Remaining columns:", hour_data.columns.tolist())

### 1.3 Holiday and Working Day Analysis
Investigating how holidays and working days affect rental patterns.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Holiday analysis at hourly level
holiday_labels = {0: 'Non-Holiday', 1: 'Holiday'}
sns.boxplot(x='holiday', y='cnt', data=hour_data, ax=ax1)
ax1.set_title('Hourly Rentals: Holiday vs Non-Holiday')
ax1.set_xticklabels([holiday_labels[i] for i in [0, 1]])
ax1.set_xlabel('Day Type')
ax1.set_ylabel('Hourly Rentals')

# Working day analysis at hourly level
workingday_labels = {0: 'Non-Working Day', 1: 'Working Day'}
sns.boxplot(x='workingday', y='cnt', data=hour_data, ax=ax2)
ax2.set_title('Hourly Rentals: Working vs Non-Working Days')
ax2.set_xticklabels([workingday_labels[i] for i in [0, 1]])
ax2.set_xlabel('Day Type')
ax2.set_ylabel('Hourly Rentals')

plt.tight_layout()

In [ ]:
# Create day of week visualization
plt.figure(figsize=(12, 8))

# Calculate average rentals by day of week
weekday_rentals = hour_data.groupby('weekday')['cnt'].mean().reset_index()

# Map weekday numbers to names for better readability
weekday_names = {
    0: 'Sunday',
    1: 'Monday',
    2: 'Tuesday',
    3: 'Wednesday',
    4: 'Thursday',
    5: 'Friday',
    6: 'Saturday'
}
weekday_rentals['day_name'] = weekday_rentals['weekday'].map(weekday_names)

# Create bar plot with improved styling
ax = sns.barplot(x='day_name', y='cnt', data=weekday_rentals, 
                order=[weekday_names[i] for i in range(7)],
                palette='Blues_d')
plt.title('Average Hourly Bike Rentals by Day of Week', fontsize=16, pad=20)
plt.xlabel('Day of Week', fontsize=14)
plt.ylabel('Average Hourly Rentals', fontsize=14)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Add value labels on top of bars
for p in ax.patches:
    ax.annotate(f'{p.get_height():.1f}', 
               (p.get_x() + p.get_width() / 2., p.get_height()),
               ha='center', va='bottom',
               xytext=(0, 5),
               textcoords='offset points')

# Add weekday vs weekend comparison
weekday_mask = hour_data['weekday'].isin([1, 2, 3, 4, 5])
weekend_mask = hour_data['weekday'].isin([0, 6])
weekday_avg = hour_data[weekday_mask]['cnt'].mean()
weekend_avg = hour_data[weekend_mask]['cnt'].mean()
diff_pct = ((weekday_avg - weekend_avg) / weekend_avg) * 100

In [ ]:
# Analyze weekday effect on bike rentals (cnt) with hourly patterns by day
plt.figure(figsize=(14, 10))

# Create hourly patterns by day of week
hourly_weekday_data = hour_data.pivot_table(
    values='cnt', 
    index='hr', 
    columns='weekday', 
    aggfunc='mean'
)

# Map weekday numbers to names for better readability
hourly_weekday_data.columns = [weekday_names[day] for day in hourly_weekday_data.columns]

# Plot hourly patterns by day of week
for day in hourly_weekday_data.columns:
    plt.plot(hourly_weekday_data.index, hourly_weekday_data[day], 
             label=day, marker='o', markersize=5, alpha=0.7)

# Add highlights for peak hours
plt.axvspan(7, 9, color='lightblue', alpha=0.3, label='Morning Peak')
plt.axvspan(16, 19, color='lightyellow', alpha=0.3, label='Evening Peak')

plt.title('Hourly Rental Patterns by Day of Week', fontsize=16)
plt.xlabel('Hour of Day', fontsize=12)
plt.ylabel('Average Rentals', fontsize=12)
plt.xticks(range(0, 24))
plt.grid(linestyle='--', alpha=0.7)
plt.legend(title='Day of Week', loc='upper center', bbox_to_anchor=(0.5, -0.15), 
           fancybox=True, shadow=True, ncol=4)

# Calculate and display correlation between weekday and cnt
weekday_cnt_corr = hour_data['weekday'].corr(hour_data['cnt'])
plt.annotate(f'Correlation between weekday and cnt: {weekday_cnt_corr:.3f}',
            xy=(0.5, 0.97), xycoords='axes fraction',
            ha='center', fontsize=12,
            bbox=dict(boxstyle='round,pad=0.5', fc='white', alpha=0.8))

plt.tight_layout()
plt.show()

Due to the spread of rentals on weekday vs weekend, we will drop the 'weekday' column as it is less correlated with 'cnt' than 'workingday'.

In [ ]:
# Drop the weekday column as it's less correlated with 'cnt' than 'workingday'
hour_data = hour_data.drop(columns=['weekday'])
print(f"'weekday' column dropped. Remaining columns: {hour_data.columns.tolist()}")

### 1.4 Weather Impact Analysis
Analyzing how different weather conditions affect rental behavior.

In [ ]:
# Define weather situation mapping
weather_labels = {
    1: 'Clear/Partly Cloudy',
    2: 'Mist/Cloudy',
    3: 'Light Rain/Snow',
    4: 'Heavy Rain/Snow'
}

# Temperature vs Count with improved styling
plt.figure(figsize=(10, 6))
sns.scatterplot(x='temp', y='cnt', data=hour_data, alpha=0.5)
plt.title('Impact of Temperature on Bike Rentals', pad=15)
plt.xlabel('Temperature (Normalized 0-1 scale)')
plt.ylabel('Number of Hourly Rentals')

# Add trend line
sns.regplot(x='temp', y='cnt', data=hour_data, scatter=False, color='red')

plt.tight_layout()
plt.show()

# Weather Situation vs Count with descriptive labels
plt.figure(figsize=(12, 6))
sns.boxplot(x='weathersit', y='cnt', data=hour_data)
plt.title('Impact of Weather Conditions on Bike Rentals', pad=15)
plt.xlabel('Weather Condition')
plt.ylabel('Number of Hourly Rentals')

# Update x-axis labels with weather descriptions
plt.xticks(range(len(weather_labels)), 
          [weather_labels[i] for i in range(1, 5)], 
          rotation=45)

plt.tight_layout()
plt.show()

### 1.5 Feature Correlation Analysis
Examining relationships between numerical features.

In [ ]:
feature_labels = {
    'cnt': 'Total Rentals',
    'temp': 'Temperature',
    'atemp': 'Air Temp',
    'hum': 'Humidity',
    'windspeed': 'Wind Speed',
    'hr': 'Hour',
    'season': 'Season'
}

correlation_features = ['cnt', 'temp', 'atemp', 'hum', 'windspeed', 'hr', 'season']
correlation_matrix = hour_data[correlation_features].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, 
            annot=True, 
            cmap='coolwarm', 
            center=0,
            xticklabels=[feature_labels[col] for col in correlation_features],
            yticklabels=[feature_labels[col] for col in correlation_features])
plt.title('Correlation Matrix of Numerical Features')

In [ ]:
# Calculate rental distribution statistics
print("1. Distribution Statistics:")
print(f"Mean rentals: {hour_data['cnt'].mean():.2f}")
print(f"Median rentals: {hour_data['cnt'].median():.2f}")
print(f"Skewness: {hour_data['cnt'].skew():.2f}")

# Calculate peak hours statistics
hourly_avg = hour_data.groupby('hr')['cnt'].mean()
peak_hours = hourly_avg.nlargest(3)
print("\n2. Peak Hours:")
print(peak_hours)

# Calculate weather correlations
print("\n3. Weather Correlations:")
print(f"Temperature correlation: {hour_data['cnt'].corr(hour_data['temp']):.2f}")
print(f"Humidity correlation: {hour_data['cnt'].corr(hour_data['hum']):.2f}")
print(f"Wind speed correlation: {hour_data['cnt'].corr(hour_data['windspeed']):.2f}")

We will drop the 'instant' and 'dteday' columns as they are not needed for our analysis. The 'instant' column is an index, and 'dteday' is a date column that we will not use in our analysis.
'casual' and 'registered' columns are also dropped as they are only gathered after bike rental. We will only use the 'cnt' column as our target variable.

In [ ]:
# Drop index-like or redundant columns
columns_to_drop = ['instant', 'dteday', 'casual', 'registered']

# Drop columns and create clean dataset
hour_data_clean = hour_data.drop(columns=columns_to_drop)

print("\nColumns dropped:", columns_to_drop)
print("Remaining columns:", hour_data_clean.columns.tolist())

# Update our working dataset
hour_data = hour_data_clean

In [ ]:
# Calculate correlation between temp and atemp
temp_correlation = hour_data['temp'].corr(hour_data['atemp'])

# Create visualization to show relationship
plt.figure(figsize=(8, 6))
sns.scatterplot(data=hour_data, x='temp', y='atemp', alpha=0.5)
plt.title(f'Temperature vs Apparent Temperature\nCorrelation: {temp_correlation:.3f}')
plt.xlabel('Temperature')
plt.ylabel('Apparent Temperature (Feels Like)')

# Print analysis
print(f"Correlation between temp and atemp: {temp_correlation:.3f}")

# Check their individual correlations with the target variable
temp_target_corr = hour_data['temp'].corr(hour_data['cnt'])
atemp_target_corr = hour_data['atemp'].corr(hour_data['cnt'])

print("\nCorrelations with rental count (cnt):")
print(f"Temperature: {temp_target_corr:.3f}")
print(f"Apparent Temperature: {atemp_target_corr:.3f}")

I will dorp the 'temp' column as it is less correlated with 'cnt' than 'atemp'.

In [ ]:
hour_data = hour_data.drop(columns=['atemp'])
print("Columns remaining after dropping 'atemp':", hour_data.columns.tolist())

In [ ]:
# Drop any remaining problematic columns
df_clean = hour_data
del hour_data

print("Final columns in cleaned dataset:", df_clean.columns.tolist())


## 2. Data Splitting


We will split our data into three sets:
- Training set (60%): Used to train the model
- Validation set (20%): Used to tune hyperparameters and evaluate model during training
- Test set (20%): Used for final model evaluation

In [ ]:
# First, make sure the date column is properly formatted
import pandas as pd

# Add a helper column if not already present to sort chronologically
df_clean['datetime'] = pd.to_datetime(df_clean['dteday']) + pd.to_timedelta(df_clean['hr'], unit='h')

# Sort by datetime to ensure chronological order
df_clean = df_clean.sort_values('datetime').reset_index(drop=True)

# Create the splits based on time
total_rows = len(df_clean)
train_end = int(total_rows * 0.6)
val_end = int(total_rows * 0.8)

# Create train, validation, and test datasets
X = df_clean.drop('cnt', axis=1)
y = df_clean['cnt']

X_train = X.iloc[:train_end]
y_train = y.iloc[:train_end]

X_val = X.iloc[train_end:val_end]
y_val = y.iloc[train_end:val_end]

X_test = X.iloc[val_end:]
y_test = y.iloc[val_end:]

print(f"Training set size: {X_train.shape[0]} ({len(X_train)/len(X):.1%})")
print(f"Validation set size: {X_val.shape[0]} ({len(X_val)/len(X):.1%})")
print(f"Test set size: {X_test.shape[0]} ({len(X_test)/len(X):.1%})")

# Visualize the time periods for each split
plt.figure(figsize=(12, 6))
plt.plot(df_clean['datetime'], df_clean['cnt'], color='lightgray', alpha=0.5, label='All Data')
plt.plot(X_train['datetime'], y_train, color='blue', alpha=0.7, label='Training Set')
plt.plot(X_val['datetime'], y_val, color='green', alpha=0.7, label='Validation Set')
plt.plot(X_test['datetime'], y_test, color='red', alpha=0.7, label='Test Set')
plt.title('Rolling Anchor Time Series Split')
plt.xlabel('Date')
plt.ylabel('Bike Rentals')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Create a pie chart showing the data split proportions
import matplotlib.pyplot as plt

# Calculate percentages
total = len(X)
train_pct = len(X_train) / total * 100
val_pct = len(X_val) / total * 100 
test_pct = len(X_test) / total * 100

# Create pie chart
plt.figure(figsize=(2, 2))
plt.pie([train_pct, val_pct, test_pct], 
        labels=['Training Set', 'Validation Set', 'Test Set'],
        autopct='%1.1f%%',
        colors=['#2ecc71', '#3498db', '#e74c3c'])
plt.title('Dataset Split Distribution')
plt.axis('equal')
plt.show()


## 3. Feature Engineering

In [ ]:
from scipy.stats import chi2_contingency

# Create cross-tabulation
workingday_weekday_crosstab = pd.crosstab(hour_data['workingday'], hour_data['weekday'])

# Perform chi-square test
chi2, p, dof, expected = chi2_contingency(workingday_weekday_crosstab)

print(f"Chi2: {chi2:.2f}")
print(f"p-value: {p:.10f}")
print(f"DOF: {dof}")
print(f"Hypothesis: {'Rejected' if p < 0.05 else 'Not rejected'}")

# Working vs non-working day average comparison
working_avg = hour_data[hour_data['workingday'] == 1]['cnt'].mean()
non_working_avg = hour_data[hour_data['workingday'] == 0]['cnt'].mean()
diff_pct = ((working_avg - non_working_avg) / non_working_avg) * 100
print(f"Working day rentals: {working_avg:.2f}")
print(f"Non-working day rentals: {non_working_avg:.2f}")
print(f"Difference: {diff_pct:.1f}%")

Due to Workingday and Weekday being somewhat correlated, I will be one-hot encoding the Weekday variable to avoid multicollinearity issues. This will allow us to analyze the impact of each weekday on bike rentals while keeping the Workingday variable intact.


In [ ]:
# Create a simplified binary day_type feature (1 = workday, 0 = weekend/holiday)
def categorize_binary_day_type(row):
    if row['holiday'] == 1 or (row['workingday'] == 0 and row['holiday'] == 0):
        return 0  # Holiday or weekend
    else:
        return 1  # Regular working weekday

# Apply this to the hour_data dataframe
hour_data['is_workday'] = hour_data.apply(categorize_binary_day_type, axis=1)

# Display the distribution of binary day types
binary_day_counts = hour_data['is_workday'].value_counts()
print("Distribution of binary day types:")
print(f"Non-workdays (0): {binary_day_counts.get(0, 0)}")
print(f"Workdays (1): {binary_day_counts.get(1, 0)}")

# Drop the now-redundant columns
hour_data = hour_data.drop(columns=['holiday', 'workingday', 'weekday'])
print("Columns after dropping redundant day features:", hour_data.columns.tolist())

I will be one-hot encoding the weather variable as well to capture the impact of different weather conditions on bike rentals. This will allow us to analyze the effect of each weather condition while keeping the original variable intact.

In [ ]:
# One-hot encode the weathersit column using pandas get_dummies
weather_encoded = pd.get_dummies(hour_data['weathersit'], prefix='weathersit', drop_first=False)

# Display the first few rows of the encoded data
print("Weather condition encoded features:")
print(weather_encoded.head())

# Add the encoded columns to the original dataframe
hour_data = pd.concat([hour_data, weather_encoded], axis=1)

# Drop the original weathersit column
hour_data = hour_data.drop(columns=['weathersit'])

print("\nColumns after encoding weathersit:", hour_data.columns.tolist())

We will categorize 'atemp', 'hum' and 'windspeed' as numeric features.
We will categorize 'season', 'mnth', 'holiday', 'workingday' and 'weather' as categorical features.
We will categorize 'hr' and 'weekday' as cyclic features to capture the cyclical nature of time.

For the cyclical features we will use sine and cosine transformations to represent the cyclical nature of time. This allows us to capture the periodicity of these features in a way that is meaningful for our model.

## 4. Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Initialize and train the model
lr_model = LinearRegression()
lr_model.fit(X_train_processed, y_train)

# Make predictions
y_train_pred = lr_model.predict(X_train_processed)
y_val_pred = lr_model.predict(X_val_processed)

# Calculate metrics
metrics = {
    'MSE': mean_squared_error(y_val, y_val_pred),
    'MAE': mean_absolute_error(y_val, y_val_pred),
    'R2': r2_score(y_val, y_val_pred)
}

print("Validation Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

# Plot residuals
plt.figure(figsize=(10, 6))
residuals = y_val - y_val_pred
sns.histplot(residuals, kde=True)
plt.title('Distribution of Residuals')
plt.xlabel('Residual Value')
plt.show()

# Analyze bias and variance
print("\nBias-Variance Analysis:")
print(f"Training R2: {r2_score(y_train, y_train_pred):.4f}")
print(f"Validation R2: {r2_score(y_val, y_val_pred):.4f}")

## 5. Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Initialize and train the model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_processed, y_train)

# Make predictions
y_train_pred_rf = rf_model.predict(X_train_processed)
y_val_pred_rf = rf_model.predict(X_val_processed)

# Calculate metrics
metrics_rf = {
    'MSE': mean_squared_error(y_val, y_val_pred_rf),
    'MAE': mean_absolute_error(y_val, y_val_pred_rf),
    'R2': r2_score(y_val, y_val_pred_rf)
}

print("Random Forest Validation Metrics:")
for metric, value in metrics_rf.items():
    print(f"{metric}: {value:.4f}")

# Feature importance plot
plt.figure(figsize=(12, 8))
feature_importance = pd.Series(rf_model.feature_importances_, index=feature_names)
feature_importance.nlargest(10).plot(kind='barh')
plt.title('Top 10 Most Important Features')
plt.show()

# Compare with baseline
print("\nComparison with Linear Regression:")
for metric in metrics.keys():
    improvement = (metrics_rf[metric] - metrics[metric]) / abs(metrics[metric]) * 100
    print(f"{metric} improvement: {improvement:.2f}%")

In [ ]:
# Save the Random Forest model
import pickle

# Save the current model if tuned version isn't available yet
rf_model_path = os.path.join(checkpoint_dir, 'rf_model.pkl')
with open(rf_model_path, 'wb') as f:
    pickle.dump(rf_model, f)

print(f"Random Forest model saved to {rf_model_path}")

# We'll save the tuned version later after hyperparameter tuning

## 6. Gradient Boosting

In [ ]:
from xgboost import XGBRegressor
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
import seaborn as sns

# Start timing
start_time = time.time()

# Initialize and train the model with verbose logging
print("Training XGBoost model...")
xgb_model = XGBRegressor(
    random_state=42, 
    verbose=1,
    eval_metric=['rmse', 'mae'],
    early_stopping_rounds=10  # Move this parameter here
)
xgb_model.fit(
    X_train_processed, 
    y_train, 
    eval_set=[(X_train_processed, y_train), (X_val_processed, y_val)],
    verbose=True  # Keep this to see progress during training
)

# Calculate training time
training_time = time.time() - start_time
print(f"XGBoost training completed in {training_time:.2f} seconds\n")

# Make predictions
y_train_pred_xgb = xgb_model.predict(X_train_processed)
y_val_pred_xgb = xgb_model.predict(X_val_processed)

# Calculate comprehensive metrics
train_metrics = {
    'MSE': mean_squared_error(y_train, y_train_pred_xgb),
    'RMSE': np.sqrt(mean_squared_error(y_train, y_train_pred_xgb)),
    'MAE': mean_absolute_error(y_train, y_train_pred_xgb),
    'R2': r2_score(y_train, y_train_pred_xgb),
    'Explained Variance': explained_variance_score(y_train, y_train_pred_xgb)
}

val_metrics = {
    'MSE': mean_squared_error(y_val, y_val_pred_xgb),
    'RMSE': np.sqrt(mean_squared_error(y_val, y_val_pred_xgb)),
    'MAE': mean_absolute_error(y_val, y_val_pred_xgb),
    'R2': r2_score(y_val, y_val_pred_xgb),
    'Explained Variance': explained_variance_score(y_val, y_val_pred_xgb)
}

# Print metrics in a formatted table
print("XGBoost Performance Metrics:")
print(f"{'Metric':<20} {'Training':<15} {'Validation':<15}")
print("-" * 50)
for metric in train_metrics.keys():
    print(f"{metric:<20} {train_metrics[metric]:<15.4f} {val_metrics[metric]:<15.4f}")

# Plot feature importance
plt.figure(figsize=(12, 8))
feature_importance = pd.Series(xgb_model.feature_importances_, index=feature_names).sort_values(ascending=False)
ax = feature_importance.head(15).plot(kind='barh', 
                                     color=sns.color_palette("viridis", len(feature_importance.head(15))))
plt.title('Top 15 Most Important Features in XGBoost Model', fontsize=14)
plt.xlabel('Importance Score', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)

# Add value annotations to the bars
for i, v in enumerate(feature_importance.head(15)):
    ax.text(v + 0.01, i, f'{v:.3f}', va='center')
    
plt.tight_layout()
plt.show()

# Plot residuals
plt.figure(figsize=(10, 6))
residuals_xgb = y_val - y_val_pred_xgb
sns.histplot(residuals_xgb, kde=True, bins=30, color='steelblue')
plt.axvline(x=0, color='red', linestyle='--', linewidth=2)
plt.title('Distribution of XGBoost Residuals', fontsize=14)
plt.xlabel('Residual Value', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.grid(True, alpha=0.3)
plt.show()

# Create a scatter plot of actual vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_val, y_val_pred_xgb, alpha=0.5, color='steelblue')
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2)
plt.title('XGBoost: Actual vs Predicted Values', fontsize=14)
plt.xlabel('Actual', fontsize=12)
plt.ylabel('Predicted', fontsize=12)
plt.grid(True, alpha=0.3)
plt.show()

# Compare with previous models
print("\nComparison with Previous Models:")
for metric in ['MSE', 'MAE', 'R2']:
    improvement_rf = (val_metrics[metric] - metrics_rf[metric]) / abs(metrics_rf[metric]) * 100
    improvement_lr = (val_metrics[metric] - metrics[metric]) / abs(metrics[metric]) * 100
    
    # Display with colored text for better visualization
    rf_status = "✓ Better" if ((metric == 'R2' and improvement_rf > 0) or 
                               (metric != 'R2' and improvement_rf < 0)) else "✗ Worse"
    lr_status = "✓ Better" if ((metric == 'R2' and improvement_lr > 0) or 
                              (metric != 'R2' and improvement_lr < 0)) else "✗ Worse"
    
    print(f"{metric} compared to Random Forest: {improvement_rf:.2f}% ({rf_status})")
    print(f"{metric} compared to Linear Regression: {improvement_lr:.2f}% ({lr_status})")

## 7. Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import logging
from datetime import datetime

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('hyperparameter_tuning')

# Define parameter distributions with more detailed logging
logger.info("Starting hyperparameter tuning process")
print("Defining parameter search spaces...")
rf_param_grid = {
    'n_estimators': randint(50, 200),
    'max_depth': randint(5, 30),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'max_features': [0.5, 0.7, 1.0, 'sqrt', 'log2', None]
}
logger.info(f"Random Forest parameters to tune: {', '.join(rf_param_grid.keys())}")
print("Random Forest parameters to tune:", ", ".join(rf_param_grid.keys()))

xgb_param_grid = {
    'learning_rate': uniform(0.01, 0.3),
    'n_estimators': randint(50, 200),
    'max_depth': randint(3, 10),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'gamma': uniform(0, 1),
    'min_child_weight': randint(1, 10)
}
logger.info(f"XGBoost parameters to tune: {', '.join(xgb_param_grid.keys())}")
print("XGBoost parameters to tune:", ", ".join(xgb_param_grid.keys()))

# Custom callback for logging during random search
class LoggingCallback:
    def __init__(self, model_name):
        self.model_name = model_name
        self.iteration = 0
        self.start_time = time.time()
        
    def __call__(self, model, params):
        self.iteration += 1
        elapsed = time.time() - self.start_time
        logger.info(f"{self.model_name} - Iteration {self.iteration} completed in {elapsed:.2f}s")
        logger.info(f"Current parameters: {params}")
        
        # Every 10 iterations, print a summary
        if self.iteration % 10 == 0:
            print(f"{self.model_name} - Completed {self.iteration} iterations in {elapsed:.2f}s")

# Random Forest Tuning with improved logging
print("\n" + "="*50)
print("Starting Random Forest hyperparameter tuning...")
logger.info("Starting Random Forest hyperparameter tuning")
start_time = time.time()

# Create callback for RF model
rf_callback = LoggingCallback("Random Forest")

rf_random_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42),
    param_distributions=rf_param_grid,
    n_iter=50,
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    verbose=2,
    n_jobs=-1  # Use all available cores
)

logger.info(f"Starting RF RandomizedSearchCV with {rf_random_search.n_iter} iterations")
print(f"Running {rf_random_search.n_iter} iterations of Random Forest hyperparameter search...")

try:
    # Fit the model - we'll wrap this in a try-except to provide graceful error handling
    rf_random_search.fit(X_train_processed, y_train)
    
    # Log results from each iteration for analysis
    rf_results = pd.DataFrame(rf_random_search.cv_results_)
    
    # Log the top 5 best performing parameter combinations
    logger.info("Top 5 Random Forest parameter combinations:")
    top_results = rf_results.sort_values('mean_test_score', ascending=False).head(5)
    for i, row in top_results.iterrows():
        logger.info(f"Rank {i+1}: Score = {-row['mean_test_score']:.4f}, Params = {row['params']}")
        
except Exception as e:
    logger.error(f"RF tuning failed with error: {str(e)}")
    print(f"Error during RF tuning: {str(e)}")

rf_tuning_time = time.time() - start_time
logger.info(f"Random Forest tuning completed in {rf_tuning_time:.2f} seconds")
print(f"\nRandom Forest tuning completed in {rf_tuning_time:.2f} seconds")
print(f"Best MSE: {-rf_random_search.best_score_:.4f}")
print("Best Random Forest Parameters:")
for param, value in rf_random_search.best_params_.items():
    print(f"    {param}: {value}")
    logger.info(f"Best {param}: {value}")

# Create a bar plot of top parameter importance
rf_results = pd.DataFrame(rf_random_search.cv_results_)
rf_results = rf_results.sort_values('mean_test_score', ascending=False)
logger.info(f"Sorted {len(rf_results)} results by performance")

# XGBoost Tuning with improved logging
print("\n" + "="*50)
print("Starting XGBoost hyperparameter tuning...")
logger.info("Starting XGBoost hyperparameter tuning")
start_time = time.time()

# Create callback for XGB model
xgb_callback = LoggingCallback("XGBoost")

xgb_random_search = RandomizedSearchCV(
    XGBRegressor(random_state=42),
    param_distributions=xgb_param_grid,
    n_iter=50,
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    verbose=2,
    n_jobs=-1
)

logger.info(f"Starting XGB RandomizedSearchCV with {xgb_random_search.n_iter} iterations")
print(f"Running {xgb_random_search.n_iter} iterations of XGBoost hyperparameter search...")

try:
    # Fit the model
    xgb_random_search.fit(X_train_processed, y_train)
    
    # Log results from each iteration for analysis
    xgb_results = pd.DataFrame(xgb_random_search.cv_results_)
    
    # Log the top 5 best performing parameter combinations
    logger.info("Top 5 XGBoost parameter combinations:")
    top_results = xgb_results.sort_values('mean_test_score', ascending=False).head(5)
    for i, row in top_results.iterrows():
        logger.info(f"Rank {i+1}: Score = {-row['mean_test_score']:.4f}, Params = {row['params']}")
        
except Exception as e:
    logger.error(f"XGB tuning failed with error: {str(e)}")
    print(f"Error during XGB tuning: {str(e)}")

xgb_tuning_time = time.time() - start_time
logger.info(f"XGBoost tuning completed in {xgb_tuning_time:.2f} seconds")
print(f"\nXGBoost tuning completed in {xgb_tuning_time:.2f} seconds")
print(f"Best MSE: {-xgb_random_search.best_score_:.4f}")
print("Best XGBoost Parameters:")
for param, value in xgb_random_search.best_params_.items():
    print(f"    {param}: {value}")
    logger.info(f"Best {param}: {value}")

# Get the best models
tuned_rf = rf_random_search.best_estimator_
tuned_xgb = xgb_random_search.best_estimator_

# Save detailed tuning results to CSV for later analysis
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
rf_results.to_csv(f"{checkpoint_dir}/rf_tuning_results_{timestamp}.csv", index=False)
xgb_results.to_csv(f"{checkpoint_dir}/xgb_tuning_results_{timestamp}.csv", index=False)
logger.info(f"Tuning results saved to CSV files in {checkpoint_dir}")

# Create a function for detailed model evaluation
def evaluate_model(model, X_train, X_val, y_train, y_val, model_name):
    logger.info(f"Evaluating {model_name} model")
    eval_start_time = time.time()
    
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    
    # Calculate comprehensive metrics
    train_metrics = {
        'MSE': mean_squared_error(y_train, y_train_pred),
        'RMSE': np.sqrt(mean_squared_error(y_train, y_train_pred)),
        'MAE': mean_absolute_error(y_train, y_train_pred),
        'R2': r2_score(y_train, y_train_pred),
        'Explained Variance': explained_variance_score(y_train, y_train_pred)
    }
    
    val_metrics = {
        'MSE': mean_squared_error(y_val, y_val_pred),
        'RMSE': np.sqrt(mean_squared_error(y_val, y_val_pred)),
        'MAE': mean_absolute_error(y_val, y_val_pred),
        'R2': r2_score(y_val, y_val_pred),
        'Explained Variance': explained_variance_score(y_val, y_val_pred)
    }
    
    # Log all metrics
    logger.info(f"{model_name} - Train metrics: {train_metrics}")
    logger.info(f"{model_name} - Validation metrics: {val_metrics}")
    
    # Print metrics in a formatted table
    print(f"\n{model_name} Performance Metrics:")
    print(f"{'Metric':<20} {'Training':<15} {'Validation':<15}")
    print("-" * 50)
    for metric in train_metrics.keys():
        print(f"{metric:<20} {train_metrics[metric]:<15.4f} {val_metrics[metric]:<15.4f}")
    
    eval_time = time.time() - eval_start_time
    logger.info(f"Model evaluation completed in {eval_time:.2f} seconds")
    
    # Return metrics for further comparison
    return train_metrics, val_metrics, y_val_pred

# Evaluate the tuned models
print("\nEvaluating tuned models...")
logger.info("Starting evaluation of tuned models")
rf_train_metrics, rf_val_metrics, y_val_pred_tuned_rf = evaluate_model(
    tuned_rf, X_train_processed, X_val_processed, y_train, y_val, "Tuned Random Forest")

xgb_train_metrics, xgb_val_metrics, y_val_pred_tuned_xgb = evaluate_model(
    tuned_xgb, X_train_processed, X_val_processed, y_train, y_val, "Tuned XGBoost")

# Compare models and log differences
for metric in ['MSE', 'MAE', 'R2']:
    diff = ((xgb_val_metrics[metric] - rf_val_metrics[metric]) / abs(rf_val_metrics[metric])) * 100
    logger.info(f"XGBoost vs RF {metric} difference: {diff:.2f}%")

# Compare the tuned models visually
metrics_to_compare = ['MSE', 'MAE', 'R2']
plt.figure(figsize=(15, 6))
logger.info("Creating comparison visualizations")

# Create bar chart to compare models
x = np.arange(len(metrics_to_compare))
width = 0.35

# Plot validation metrics
plt.subplot(1, 2, 1)
plt.bar(x - width/2, [rf_val_metrics[m] for m in metrics_to_compare], width, label='Random Forest')
plt.bar(x + width/2, [xgb_val_metrics[m] for m in metrics_to_compare], width, label='XGBoost')
plt.xlabel('Metrics')
plt.ylabel('Value')
plt.title('Validation Metrics Comparison')
plt.xticks(x, metrics_to_compare)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Plot residuals distribution for both models
plt.subplot(1, 2, 2)
rf_residuals = y_val - y_val_pred_tuned_rf
xgb_residuals = y_val - y_val_pred_tuned_xgb
sns.kdeplot(rf_residuals, label='Random Forest', fill=True, alpha=0.3)
sns.kdeplot(xgb_residuals, label='XGBoost', fill=True, alpha=0.3)
plt.axvline(x=0, color='black', linestyle='--')
plt.title('Residuals Distribution')
plt.xlabel('Residual Value')
plt.ylabel('Density')
plt.legend()
plt.grid(linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig(f"{checkpoint_dir}/model_comparison_{timestamp}.png")
logger.info(f"Comparison visualization saved to {checkpoint_dir}/model_comparison_{timestamp}.png")
plt.show()

logger.info("Hyperparameter tuning and evaluation complete")

In [ ]:
# Save the best XGBoost model
import pickle

# Save the tuned XGBoost model
xgb_model_path = os.path.join(checkpoint_dir, 'xgb_tuned_model.pkl')
with open(xgb_model_path, 'wb') as f:
    pickle.dump(tuned_xgb, f)

print(f"Tuned XGBoost model saved to {xgb_model_path}")

# Function to load the model if needed
def load_xgb_model(model_path=None):
    if model_path is None:
        model_path = os.path.join(checkpoint_dir, 'xgb_tuned_model.pkl')
    with open(model_path, 'rb') as f:
        return pickle.load(f)

## 8. Evalutaion/Refinement

In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
import seaborn as sns

print("="*80)
print("MODEL REFINEMENT AND FEATURE ENGINEERING")
print("="*80)

# Log feature engineering process
print("Adding interaction terms based on EDA insights...")
X_train_interactions = X_train_processed.copy()
X_val_interactions = X_val_processed.copy()
X_test_interactions = X_test_processed.copy()

# Document the original feature space
print(f"Original feature space: {X_train_processed.shape[1]} features")

# 1. Add interaction between temperature and humidity
print("Adding temp × humidity interaction feature")
X_train_interactions['atemp_hum_interaction'] = X_train_interactions['atemp'] * X_train_interactions['hum']
X_val_interactions['atemp_hum_interaction'] = X_val_interactions['atemp'] * X_val_interactions['hum']
X_test_interactions['atemp_hum_interaction'] = X_test_interactions['atemp'] * X_test_interactions['hum']

# 2. Add additional interaction for time of day and weather
print("Adding hour (cyclical) × weather interaction features")
X_train_interactions['hr_sin_weather'] = X_train_interactions['hr_sin'] * X_train_interactions['weathersit_1']
X_val_interactions['hr_sin_weather'] = X_val_interactions['hr_sin'] * X_val_interactions['weathersit_1']
X_test_interactions['hr_sin_weather'] = X_test_interactions['hr_sin'] * X_test_interactions['weathersit_1']

# 3. Add season-related interactions
print("Adding season × temperature interaction")
for season_col in [col for col in X_train_interactions.columns if 'season' in col]:
    X_train_interactions[f'{season_col}_temp'] = X_train_interactions[season_col] * X_train_interactions['atemp']
    X_val_interactions[f'{season_col}_temp'] = X_val_interactions[season_col] * X_val_interactions['atemp']
    X_test_interactions[f'{season_col}_temp'] = X_test_interactions[season_col] * X_test_interactions['atemp']

# Log the expanded feature space
print(f"Expanded feature space: {X_train_interactions.shape[1]} features")
print(f"Added {X_train_interactions.shape[1] - X_train_processed.shape[1]} new interaction features")

# Function to evaluate model with detailed metrics
def evaluate_refined_model(model, X_train, X_val, y_train, y_val, model_name):
    # Start timing
    start_time = time.time()
    
    # Train the model
    print(f"\nTraining refined {model_name} model...")
    model.fit(X_train, y_train)
    
    # Calculate training time
    training_time = time.time() - start_time
    print(f"Training completed in {training_time:.2f} seconds")
    
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    
    # Calculate comprehensive metrics
    train_metrics = {
        'MSE': mean_squared_error(y_train, y_train_pred),
        'RMSE': np.sqrt(mean_squared_error(y_train, y_train_pred)),
        'MAE': mean_absolute_error(y_train, y_train_pred),
        'R2': r2_score(y_train, y_train_pred),
        'Explained Variance': explained_variance_score(y_train, y_train_pred)
    }
    
    val_metrics = {
        'MSE': mean_squared_error(y_val, y_val_pred),
        'RMSE': np.sqrt(mean_squared_error(y_val, y_val_pred)),
        'MAE': mean_absolute_error(y_val, y_val_pred),
        'R2': r2_score(y_val, y_val_pred),
        'Explained Variance': explained_variance_score(y_val, y_val_pred)
    }
    
    # Print metrics in a formatted table
    print(f"\n{model_name} Performance Metrics:")
    print(f"{'Metric':<20} {'Training':<15} {'Validation':<15}")
    print("-" * 50)
    for metric in train_metrics.keys():
        print(f"{metric:<20} {train_metrics[metric]:<15.4f} {val_metrics[metric]:<15.4f}")
    
    return train_metrics, val_metrics, y_val_pred

# Evaluate the refined XGBoost model
refined_xgb = tuned_xgb
_, refined_metrics, y_val_pred_refined = evaluate_refined_model(
    refined_xgb, X_train_interactions, X_val_interactions, y_train, y_val, "Refined XGBoost")

# Compare with baseline tuned model
print("\nIMPROVEMENT ANALYSIS:")
print("-" * 50)
for metric in ['MSE', 'RMSE', 'MAE', 'R2', 'Explained Variance']:
    if metric in xgb_val_metrics:
        baseline = xgb_val_metrics[metric]
        refined = refined_metrics[metric]
        change = (refined - baseline) / abs(baseline) * 100
        
        # Determine if the change is positive or negative
        if (metric == 'R2' or metric == 'Explained Variance') and change > 0:
            status = "✓ Improved"
        elif (metric != 'R2' and metric != 'Explained Variance') and change < 0:
            status = "✓ Improved"
        else:
            status = "✗ Worsened"
        
        print(f"{metric}: {baseline:.4f} → {refined:.4f} ({change:.2f}%) {status}")

# Visualize the impact of feature engineering
plt.figure(figsize=(12, 10))

# Plot 1: Feature importance from the refined model
plt.subplot(2, 1, 1)
feature_importance = pd.Series(refined_xgb.feature_importances_, 
                               index=X_train_interactions.columns).sort_values(ascending=False)
top_features = feature_importance.head(15)
sns.barplot(x=top_features.values, y=top_features.index, palette='viridis')
plt.title('Top 15 Features After Refinement', fontsize=14)
plt.xlabel('Importance', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)

# Plot 2: Actual vs Predicted values with the refined model
plt.subplot(2, 1, 2)
plt.scatter(y_val, y_val_pred_refined, alpha=0.5, color='steelblue')
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2)
plt.title('Refined Model: Actual vs Predicted Values', fontsize=14)
plt.xlabel('Actual', fontsize=12)
plt.ylabel('Predicted', fontsize=12)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Check if any of the new interaction features appear in the top 15 important features
new_features = [col for col in X_train_interactions.columns if col not in X_train_processed.columns]
important_new_features = [feat for feat in top_features.index if feat in new_features]
print("\nIMPORTANT NEW INTERACTION FEATURES:")
for i, feat in enumerate(important_new_features, 1):
    importance = feature_importance[feat]
    rank = list(feature_importance.index).index(feat) + 1
    print(f"{i}. {feat} (Rank: {rank}, Importance: {importance:.4f})")

## 9. Final Model

In [ ]:
# Combine training and validation sets
X_final_train = pd.concat([X_train_interactions, X_val_interactions])
y_final_train = pd.concat([y_train, y_val])

# Train final model on combined data
final_model = tuned_xgb
final_model.fit(X_final_train, y_final_train)

# Evaluate on test set
y_test_pred = final_model.predict(X_test_interactions)

print("\nFinal Model Test Performance:")
print(f"MSE: {mean_squared_error(y_test, y_test_pred):.4f}")
print(f"MAE: {mean_absolute_error(y_test, y_test_pred):.4f}")
print(f"R2: {r2_score(y_test, y_test_pred):.4f}")

# Feature importance of final model
plt.figure(figsize=(12, 8))
final_importance = pd.Series(final_model.feature_importances_, index=X_final_train.columns)
final_importance.nlargest(10).plot(kind='barh')
plt.title('Top 10 Most Important Features in Final Model')
plt.show()

# Compare with validation performance
print("\nPerformance Comparison:")
print("Validation R2:", r2_score(y_val, y_val_pred_refined))
print("Test R2:", r2_score(y_test, y_test_pred))

In [ ]:
# Calculate additional statistics for model predictions
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error

# Create analysis for test predictions
print("="*50)
print("DETAILED MODEL PREDICTION ANALYSIS")
print("="*50)

# Calculate error statistics
absolute_errors = np.abs(y_test - y_test_pred)
percentage_errors = np.abs((y_test - y_test_pred) / y_test) * 100
squared_errors = (y_test - y_test_pred) ** 2

# Basic error statistics
print("\nError Statistics:")
print(f"Mean Absolute Error (MAE): {np.mean(absolute_errors):.2f}")
print(f"Median Absolute Error: {np.median(absolute_errors):.2f}")
print(f"90th Percentile of Absolute Error: {np.percentile(absolute_errors, 90):.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {np.mean(percentage_errors):.2f}%")
print(f"Root Mean Square Error (RMSE): {np.sqrt(np.mean(squared_errors)):.2f}")

# Calculate prediction statistics
print("\nPrediction Statistics:")
print(f"Mean of Actual Values: {np.mean(y_test):.2f}")
print(f"Mean of Predicted Values: {np.mean(y_test_pred):.2f}")
print(f"Standard Deviation of Actual Values: {np.std(y_test):.2f}")
print(f"Standard Deviation of Predicted Values: {np.std(y_test_pred):.2f}")
print(f"Min of Actual Values: {np.min(y_test):.2f}")
print(f"Min of Predicted Values: {np.min(y_test_pred):.2f}")
print(f"Max of Actual Values: {np.max(y_test):.2f}")
print(f"Max of Predicted Values: {np.max(y_test_pred):.2f}")

# Calculate over/under prediction statistics
over_predictions = y_test_pred > y_test
under_predictions = y_test_pred < y_test
exact_predictions = y_test_pred == y_test

print("\nOver/Under Prediction Analysis:")
print(f"Over-predictions: {np.sum(over_predictions)} ({np.mean(over_predictions)*100:.1f}%)")
print(f"Under-predictions: {np.sum(under_predictions)} ({np.mean(under_predictions)*100:.1f}%)")
print(f"Exact predictions: {np.sum(exact_predictions)} ({np.mean(exact_predictions)*100:.1f}%)")

# Calculate error distribution by actual value ranges
def error_by_range(y_true, y_pred, bin_edges):
    bin_indices = np.digitize(y_true, bin_edges) - 1
    bin_count = len(bin_edges) - 1
    bin_mae = np.zeros(bin_count)
    bin_mape = np.zeros(bin_count)
    bin_counts = np.zeros(bin_count)
    
    for i in range(bin_count):
        mask = bin_indices == i
        if np.sum(mask) > 0:
            bin_mae[i] = np.mean(np.abs(y_true[mask] - y_pred[mask]))
            bin_mape[i] = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
            bin_counts[i] = np.sum(mask)
    
    return bin_mae, bin_mape, bin_counts

# Create bin edges based on the distribution of actual values
bin_edges = np.percentile(y_test, [0, 25, 50, 75, 100])
bin_mae, bin_mape, bin_counts = error_by_range(y_test, y_test_pred, bin_edges)

print("\nError Analysis by Actual Value Range:")
print(f"{'Range':<20} {'Count':<10} {'MAE':<10} {'MAPE (%)':<10}")
print("-" * 50)
for i in range(len(bin_edges)-1):
    print(f"{bin_edges[i]:.0f}-{bin_edges[i+1]:.0f}:{' ':>10} {bin_counts[i]:<10.0f} {bin_mae[i]:<10.2f} {bin_mape[i]:<10.2f}")

# Prediction accuracy by quartile
quartiles = np.percentile(y_test, [25, 50, 75])
q1_mask = y_test <= quartiles[0]
q2_mask = (y_test > quartiles[0]) & (y_test <= quartiles[1])
q3_mask = (y_test > quartiles[1]) & (y_test <= quartiles[2])
q4_mask = y_test > quartiles[2]

print("\nPrediction Performance by Quartile:")
print(f"Q1 (≤{quartiles[0]:.2f}): R² = {r2_score(y_test[q1_mask], y_test_pred[q1_mask]):.4f}, MAE = {mean_absolute_error(y_test[q1_mask], y_test_pred[q1_mask]):.2f}")
print(f"Q2 ({quartiles[0]:.2f}-{quartiles[1]:.2f}): R² = {r2_score(y_test[q2_mask], y_test_pred[q2_mask]):.4f}, MAE = {mean_absolute_error(y_test[q2_mask], y_test_pred[q2_mask]):.2f}")
print(f"Q3 ({quartiles[1]:.2f}-{quartiles[2]:.2f}): R² = {r2_score(y_test[q3_mask], y_test_pred[q3_mask]):.4f}, MAE = {mean_absolute_error(y_test[q3_mask], y_test_pred[q3_mask]):.2f}")
print(f"Q4 (>{quartiles[2]:.2f}): R² = {r2_score(y_test[q4_mask], y_test_pred[q4_mask]):.4f}, MAE = {mean_absolute_error(y_test[q4_mask], y_test_pred[q4_mask]):.2f}")

# Visualize error distribution
plt.figure(figsize=(15, 10))

# Plot 1: Histogram of errors
plt.subplot(2, 2, 1)
errors = y_test - y_test_pred
sns.histplot(errors, kde=True, bins=30)
plt.title('Distribution of Prediction Errors', fontsize=12)
plt.xlabel('Error (Actual - Predicted)')
plt.axvline(0, color='red', linestyle='--')
plt.grid(alpha=0.3)

# Plot 2: Scatter plot of error vs actual value
plt.subplot(2, 2, 2)
plt.scatter(y_test, errors, alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Error vs Actual Value', fontsize=12)
plt.xlabel('Actual Value')
plt.ylabel('Error')
plt.grid(alpha=0.3)

# Plot 3: Box plot of absolute errors by value quartile
plt.subplot(2, 2, 3)
quartile_names = ['Q1', 'Q2', 'Q3', 'Q4']
quartile_errors = [absolute_errors[q1_mask], absolute_errors[q2_mask], 
                   absolute_errors[q3_mask], absolute_errors[q4_mask]]
plt.boxplot(quartile_errors, labels=quartile_names)
plt.title('Absolute Errors by Value Quartile', fontsize=12)
plt.ylabel('Absolute Error')
plt.grid(axis='y', alpha=0.3)

# Plot 4: Actual vs predicted with perfect prediction line
plt.subplot(2, 2, 4)
plt.scatter(y_test, y_test_pred, alpha=0.5)
min_val = min(np.min(y_test), np.min(y_test_pred))
max_val = max(np.max(y_test), np.max(y_test_pred))
plt.plot([min_val, max_val], [min_val, max_val], 'r--')
plt.title('Actual vs Predicted Values', fontsize=12)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(checkpoint_dir, 'prediction_analysis.png'))
plt.show()

# Calculate prediction metrics for specific subgroups
# For example, by season, hour of day, or weather condition
print("\nPerformance by Weather Condition:")
for weather in np.unique(X_test['weathersit']):
    weather_mask = X_test['weathersit'] == weather
    if np.sum(weather_mask) > 0:
        mae = mean_absolute_error(y_test[weather_mask], y_test_pred[weather_mask])
        r2 = r2_score(y_test[weather_mask], y_test_pred[weather_mask])
        print(f"Weather {weather}: Count = {np.sum(weather_mask)}, MAE = {mae:.2f}, R² = {r2:.4f}")

# Calculate time-based prediction performance
print("\nPerformance by Hour Range:")
morning = (X_test['hr'] >= 6) & (X_test['hr'] < 12)
afternoon = (X_test['hr'] >= 12) & (X_test['hr'] < 18)
evening = (X_test['hr'] >= 18) & (X_test['hr'] < 22)
night = (X_test['hr'] >= 22) | (X_test['hr'] < 6)

print(f"Morning (6-11): MAE = {mean_absolute_error(y_test[morning], y_test_pred[morning]):.2f}, R² = {r2_score(y_test[morning], y_test_pred[morning]):.4f}")
print(f"Afternoon (12-17): MAE = {mean_absolute_error(y_test[afternoon], y_test_pred[afternoon]):.2f}, R² = {r2_score(y_test[afternoon], y_test_pred[afternoon]):.4f}")
print(f"Evening (18-21): MAE = {mean_absolute_error(y_test[evening], y_test_pred[evening]):.2f}, R² = {r2_score(y_test[evening], y_test_pred[evening]):.4f}")
print(f"Night (22-5): MAE = {mean_absolute_error(y_test[night], y_test_pred[night]):.2f}, R² = {r2_score(y_test[night], y_test_pred[night]):.4f}")

In [ ]:
# Save the final model
final_model_path = os.path.join(checkpoint_dir, 'bike_rental_final_model.pkl')
with open(final_model_path, 'wb') as f:
    pickle.dump(final_model, f)

# Also save the feature preprocessor for future predictions
preprocessor_path = os.path.join(checkpoint_dir, 'feature_preprocessor.pkl')
with open(preprocessor_path, 'wb') as f:
    pickle.dump(preprocessor, f)

print(f"Final model saved to {final_model_path}")
print(f"Feature preprocessor saved to {preprocessor_path}")